In [14]:
import torch
from data_reading import WindowDataset
import os
import cv2
import numpy as np
from scipy import signal
import pickle

In [15]:
image_width = 128
image_height = 96
labels_folder = '../data/labels/'
imgs_folder = '../data/images/'
video_folder = '../data/training_videos/'
spectr_folder = '../data/spectrograms/'

In [16]:
all_folders_img = set([dir for dir in os.listdir(labels_folder) if 'NW' in dir or 'W' in dir])
all_folders_lab = set([dir for dir in os.listdir(imgs_folder) if 'NW' in dir or 'W' in dir])
all_folders_video = set(['Folder '+dir.split('.')[0] for dir in os.listdir(video_folder) if 'NW' in dir or 'W' in dir])
all_folders = list(all_folders_img.intersection(all_folders_lab).intersection(all_folders_video))

In [17]:
print(all_folders)

['Folder 24 W', 'Folder 29 W', 'Folder 18 NW', 'Folder 39 W', 'Folder 8 W', 'Folder 28 W', 'Folder 3 NW', 'Folder 6 NW', 'Folder 4 NW', 'Folder 19 NW', 'Folder 22 NW', 'Folder 10 W', 'Folder 31 NW', 'Folder 21 W', 'Folder 5 NW', 'Folder 17 NW', 'Folder 35 W', 'Folder 9 W', 'Folder 26 NW', 'Folder 37 W', 'Folder 2 NW', 'Folder 7 W', 'Folder 36 NW', 'Folder 30 NW', 'Folder 38 W', 'Folder 23 W', 'Folder 12 NW', 'Folder 16 NW', 'Folder 32 NW', 'Folder 20 W', 'Folder 33 W', 'Folder 25 W', 'Folder 27 W']


In [18]:
dataset = WindowDataset(video_folder, labels_folder, imgs_folder)

In [19]:
dataset.save(spectr_folder)

19.066666666666666
(8, 3, 9, 12, 559)
torch.Size([56, 9, 12, 24])
29.866666666666667
(8, 3, 9, 12, 883)
torch.Size([88, 9, 12, 24])
30.033333333333335
(8, 3, 9, 12, 888)
torch.Size([89, 9, 12, 24])
19.033333333333335
(8, 3, 9, 12, 558)
torch.Size([56, 9, 12, 24])
9.866666666666667
(8, 3, 9, 12, 283)
torch.Size([28, 9, 12, 24])
29.8
(8, 3, 9, 12, 881)
torch.Size([88, 9, 12, 24])
19.433333333333334
(8, 3, 9, 12, 570)
torch.Size([53, 9, 12, 24])
26.866666666666667
(8, 3, 9, 12, 793)
torch.Size([79, 9, 12, 24])
14.466666666666667
(8, 3, 9, 12, 421)
19.2
(8, 3, 9, 12, 563)
torch.Size([56, 9, 12, 24])
30.033333333333335
(8, 3, 9, 12, 888)
torch.Size([89, 9, 12, 24])
12.9
(8, 3, 9, 12, 374)
torch.Size([37, 9, 12, 24])
29.833333333333332
(8, 3, 9, 12, 882)
torch.Size([88, 9, 12, 24])
30.0
(8, 3, 9, 12, 887)
torch.Size([88, 9, 12, 24])
29.233333333333334
(8, 3, 9, 12, 864)
torch.Size([86, 9, 12, 24])
18.866666666666667
(8, 3, 9, 12, 553)
torch.Size([55, 9, 12, 24])
19.133333333333333
(8, 3, 9, 

In [20]:
all_spectr = {}
for file_ in all_folders:
    if 'NW' in file_ or 'W' in file_:
        file = file_[7:]+'.spectr'
        name = file.split('.')[0]
        print(f'{spectr_folder}{file}')
        with open(f'{spectr_folder}{file}','rb') as f:
            spectr = pickle.load(f)
        all_spectr[name] = spectr.permute(0,3,1,2)

../data/spectrograms/24 W.spectr
../data/spectrograms/29 W.spectr
../data/spectrograms/18 NW.spectr
../data/spectrograms/39 W.spectr
../data/spectrograms/8 W.spectr
../data/spectrograms/28 W.spectr
../data/spectrograms/3 NW.spectr
../data/spectrograms/6 NW.spectr
../data/spectrograms/4 NW.spectr
../data/spectrograms/19 NW.spectr
../data/spectrograms/22 NW.spectr
../data/spectrograms/10 W.spectr
../data/spectrograms/31 NW.spectr
../data/spectrograms/21 W.spectr
../data/spectrograms/5 NW.spectr
../data/spectrograms/17 NW.spectr
../data/spectrograms/35 W.spectr
../data/spectrograms/9 W.spectr
../data/spectrograms/26 NW.spectr
../data/spectrograms/37 W.spectr
../data/spectrograms/2 NW.spectr
../data/spectrograms/7 W.spectr
../data/spectrograms/36 NW.spectr
../data/spectrograms/30 NW.spectr
../data/spectrograms/38 W.spectr
../data/spectrograms/23 W.spectr
../data/spectrograms/12 NW.spectr
../data/spectrograms/16 NW.spectr
../data/spectrograms/32 NW.spectr
../data/spectrograms/20 W.spectr
..

In [21]:
all_images = {}
for file in all_folders:
    if 'Folder' in file:
        name = file[7:]
        print(f'{imgs_folder}{file}')
        imgs = []
        img_dir_list = os.listdir(f'{imgs_folder}{file}')
        img_dir_list.sort()
        for imgfile in img_dir_list:
            if '.jpg' in imgfile:
                img = torch.tensor(cv2.cvtColor(cv2.imread(f"{imgs_folder}{file}/{imgfile}", cv2.IMREAD_COLOR), cv2.COLOR_BGR2RGB))
                img = img.permute(2,0,1).double()
                imgs.append(img)
        imgs = torch.stack(imgs)
        all_images[name] = imgs

../data/images/Folder 24 W
../data/images/Folder 29 W
../data/images/Folder 18 NW
../data/images/Folder 39 W
../data/images/Folder 8 W
../data/images/Folder 28 W
../data/images/Folder 3 NW
../data/images/Folder 6 NW
../data/images/Folder 4 NW
../data/images/Folder 19 NW
../data/images/Folder 22 NW
../data/images/Folder 10 W
../data/images/Folder 31 NW
../data/images/Folder 21 W
../data/images/Folder 5 NW
../data/images/Folder 17 NW
../data/images/Folder 35 W
../data/images/Folder 9 W
../data/images/Folder 26 NW
../data/images/Folder 37 W
../data/images/Folder 2 NW
../data/images/Folder 7 W
../data/images/Folder 36 NW
../data/images/Folder 30 NW
../data/images/Folder 38 W
../data/images/Folder 23 W
../data/images/Folder 12 NW
../data/images/Folder 16 NW
../data/images/Folder 32 NW
../data/images/Folder 20 W
../data/images/Folder 33 W
../data/images/Folder 25 W
../data/images/Folder 27 W


In [22]:
all_labs = {}
for file in all_folders:
    if 'Folder' in file:
        name = file[7:]
        print(f'{labels_folder}{file}')
        all_targets = []
        for interfolder in os.listdir(f'{labels_folder}{file}'):
            if interfolder.endswith('YOLO'):
                lab_list_dir = os.listdir(f'{labels_folder}{file}/{interfolder}')
                lab_list_dir.sort()
                prevCount = 0
                for labfile in lab_list_dir:
                    if '.txt' in labfile:
                        count = int(labfile.split('.')[0])
                        while count > prevCount+1:
                            targets = {'boxes':torch.tensor([[]]), 'labels':torch.tensor([])}
                            all_targets.append(targets)
                            prevCount+=1
                        prevCount = count
                        boxes = []
                        labels = []
                        with open(f"{labels_folder}{file}/{interfolder}/{labfile}") as f:
                            for line in f:
                                line = line.split()
                                label, rest = int(line[0]), line[1:]
                                xcenter, ycenter, w, h = map(float, rest)
                                xcenter, ycenter = int(xcenter*image_width), int(ycenter*image_height)
                                xmin, ymin = xcenter-int(w/2*image_width), ycenter-int(h/2*image_height)
                                boxes.append(torch.tensor([int(xmin), int(ymin), int(w*image_width), int(h*image_height)]))
                                labels.append(label)
                        if len(labels) == 0:
                            targets = {'boxes':torch.tensor([[]]), 'labels':torch.tensor([])}
                        else:
                            targets = {'boxes':torch.stack(boxes).float(), 'labels':torch.tensor(labels).long()+1}
                        all_targets.append(targets)
        all_labs[name] = all_targets

../data/labels/Folder 24 W
../data/labels/Folder 29 W
../data/labels/Folder 18 NW
../data/labels/Folder 39 W
../data/labels/Folder 8 W
../data/labels/Folder 28 W
../data/labels/Folder 3 NW
../data/labels/Folder 6 NW
../data/labels/Folder 4 NW
../data/labels/Folder 19 NW
../data/labels/Folder 22 NW
../data/labels/Folder 10 W
../data/labels/Folder 31 NW
../data/labels/Folder 21 W
../data/labels/Folder 5 NW
../data/labels/Folder 17 NW
../data/labels/Folder 35 W
../data/labels/Folder 9 W
../data/labels/Folder 26 NW
../data/labels/Folder 37 W
../data/labels/Folder 2 NW
../data/labels/Folder 7 W
../data/labels/Folder 36 NW
../data/labels/Folder 30 NW
../data/labels/Folder 38 W
../data/labels/Folder 23 W
../data/labels/Folder 12 NW
../data/labels/Folder 16 NW
../data/labels/Folder 32 NW
../data/labels/Folder 20 W
../data/labels/Folder 33 W
../data/labels/Folder 25 W
../data/labels/Folder 27 W


In [23]:
with open('../data/spectro.data', 'wb') as f:
    pickle.dump(all_spectr, f)
with open('../data/images.data', 'wb') as f:
    pickle.dump(all_images, f)
with open('../data/labs.data', 'wb') as f:
    pickle.dump(all_labs, f)

In [24]:
print(all_spectr['10 W'].shape)

torch.Size([28, 24, 9, 12])


In [25]:
print(all_images['10 W'].shape)

torch.Size([30, 3, 96, 128])


In [26]:
print(len(all_labs['10 W']))
print(all_labs['10 W'][0])
# why is there a count diff?

30
{'boxes': tensor([[27.,  0., 93., 95.]]), 'labels': tensor([1])}
